# Get POI data from KOT open api

Get POI data for Seoul and put it to MongoDB (local)

In [153]:
# import libraries
import pandas as pd
import pymongo

import urllib.request as MyURL
from bs4 import BeautifulSoup
from datetime import datetime

In [154]:
# set mongodb
from pymongo import MongoClient 
client = MongoClient('localhost', 27017)

In [155]:
# set database name
db = client['cp_seoul']

In [156]:
# set collection name
collection = db['seoul']

In [4]:
# API key for data.go.kr
key = 'UiZuCz3TStJ3sDC1QBw4lZQYD%2BrwwjibhI5fSSaoGXjP3b00MseMmQ%2FNIPHLqZoVQgt8KVBAaoQ0ob7rQAMblg%3D%3D'

In [82]:
# API URL for get poi list 
#url = f'http://api.visitkorea.or.kr/openapi/service/rest/EngService/areaBasedList?ServiceKey={key}&contentTypeId=80&areaCode=1&sigunguCode=&cat1=&cat2=&cat3=&listYN=Y&MobileOS=ETC&MobileApp=TourAPI3.0_Guide&arrange=A&numOfRows=240&pageNo=1'

In [101]:
# Every place in Seoul
url = f'http://api.visitkorea.or.kr/openapi/service/rest/EngService/areaBasedList?ServiceKey={key}&contentTypeId=&areaCode=1&sigunguCode=&cat1=&cat2=&cat3=&listYN=Y&MobileOS=ETC&MobileApp=TourAPI3.0_Guide&arrange=A&numOfRows=1200&pageNo=1'

In [103]:
# get data from api server
response = MyURL.urlopen(url)

In [105]:
# parsing with BS4
results = BeautifulSoup(response, "html.parser")

In [113]:
# parse and make a document for mongo db
row = 0
for result in results.findAll('items'):
    for item in result.findAll('item'):
        row += 1
        record = {}
        for i in item:
            record[i.name] = i.contents[0]
#             print(i.name, i.contents)
#         print(record)
        collection.insert_one(record)
        
print(row)

1181


#### Get detail information for POI

In [162]:
a_record = (collection.find_one({'contentid': '1266515'}))

In [163]:
a_record

In [164]:
def get_detail(collection, contentID, key):
    url = f'http://api.visitkorea.or.kr/openapi/service/rest/EngService/detailCommon?ServiceKey={key}&contentTypeId=80&contentId=2444381&MobileOS=ETC&MobileApp=TourAPI3.0_Guide&defaultYN=Y&firstImageYN=Y&areacodeYN=Y&catcodeYN=Y&addrinfoYN=Y&mapinfoYN=Y&overviewYN=Y&transGuideYN=Y'
    response = MyURL.urlopen(url)
    results = BeautifulSoup(response, "html.parser")

    document = collection.find_one({'contentid': contentID})

    for result in results.findAll('items'):
        for item in result.findAll('item'):
            for i in item:
                document[i.name] = i.contents[0]

    return document


In [165]:
get_detail(collection, '1266515', key)

TypeError: 'NoneType' object does not support item assignment

#### Loop through for all POIs in seoul


In [168]:
cursors = collection.find()

In [169]:
print(f'start at : {datetime.now()}')
row = 0
for cursor in cursors:
    content_id = cursor['contentid']
#     new_value = {'$set': get_detail(collection, content_id, key)}
    
#     content_query = {'contentid': content_id}
#     collection.update_one(content_query, new_value)

    print(f'{row} : update {content_id} at {datetime.now()}')
    row += 1
    

print(f'end at : {datetime.now()}')

    

start at : 2019-10-18 01:15:05.763497
0 : update 2480979 at 2019-10-18 01:15:05.794672
1 : update 1240735 at 2019-10-18 01:15:05.794811
2 : update 2444381 at 2019-10-18 01:15:05.794856
3 : update 2444381 at 2019-10-18 01:15:05.794893
4 : update 2444381 at 2019-10-18 01:15:05.794928
5 : update 2444381 at 2019-10-18 01:15:05.794963
6 : update 2444381 at 2019-10-18 01:15:05.794997
7 : update 2444381 at 2019-10-18 01:15:05.795031
8 : update 2444381 at 2019-10-18 01:15:05.795065
9 : update 2444381 at 2019-10-18 01:15:05.795100
10 : update 2444381 at 2019-10-18 01:15:05.795133
11 : update 2444381 at 2019-10-18 01:15:05.795167
12 : update 2444381 at 2019-10-18 01:15:05.795202
13 : update 2444381 at 2019-10-18 01:15:05.795235
14 : update 2444381 at 2019-10-18 01:15:05.795262
15 : update 2444381 at 2019-10-18 01:15:05.795282
16 : update 2444381 at 2019-10-18 01:15:05.795300
17 : update 2444381 at 2019-10-18 01:15:05.795319
18 : update 2444381 at 2019-10-18 01:15:05.795337
19 : update 2444381 at